In [28]:
from bs4 import BeautifulSoup
import requests
import zipfile
from datetime import datetime
from io import StringIO,BytesIO
import numpy as np
import pandas as pd
DL=['20170301T08:00-0000','20170401T08:00-0000','20170501T08:00-0000','20170515T08:00-0000','20170601T08:00-0000',
     '20170701T08:00-0000','20170715T08:00-0000','20170801T08:00-0000',
     '20170815T08:00-0000','20170901T08:00-0000','20171001T08:00-0000','20171015T08:00-0000','20171101T08:00-0000','20171201T08:00-0000',
     '20180101T08:00-0000','20180201T08:00-0000','20180301T08:00-0000','20180401T08:00-0000','20180501T08:00-0000','20180515T08:00-0000',
     '20180601T08:00-0000','20180701T08:00-0000','20180715T08:00-0000','20180801T08:00-0000','20180815T08:00-0000','20180901T08:00-0000',
   '20181001T08:00-0000','20181015T08:00-0000','20181101T08:00-0000','20181201T08:00-0000','20190101T08:00-0000','20190201T08:00-0000',
  '20190301T08:00-0000','20190401T08:00-0000']


In [42]:
def load(start,end,market='ACTUAL'):
    
    start_date = datetime(int(start[:4]),int(start[4:6]),int(start[6:8]),int(start[9:11]),int(start[12:14]))
    end_date = datetime(int(end[:4]),int(end[4:6]),int(end[6:8]),int(end[9:11]),int(end[12:14]))
    Dict = {}
   
    try:
        source='http://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=SLD_FCST&market_run_id=ACTUAL&version=1&startdatetime='
        source+=start+'&enddatetime='+end
        r = requests.get(source, stream=True)
        z = zipfile.ZipFile(BytesIO(r.content))
        unzipped = [z.read(thisfile) for thisfile in z.namelist()]
        s = unzipped[0]
        l = s.decode("utf-8") 
        l=l.replace('\n',',')
        l=l.split(',')[:-1]
        L = np.array(l).reshape(int(len(l)/14),14)
        Areas = set(L[1:,7])

        for area in Areas:
            ind=(L[:,7]==area)
            if len(Dict.keys())==0:
                Dict['Timestamp']=L[ind,0]
        
            load = L[ind,11]
            Dict.update({area:load}) 
            #print(len(load))
            
        
    except:
        pass
    #print(Dict)
    #df = pd.DataFrame(data=Dict)
    return Dict
        


In [43]:
l=[]
import time
for i in range(len(DL)-1):
    print(i)
    Dict_=load(DL[i],DL[i+1])
    Cleaned_dict = {}
    Cleaned_dict['Timestamp']=Dict_['Timestamp']
    for e in Dict_.keys():
        if len(Dict_[e])==len(Dict_['Timestamp']) and e!='Timestamp':
            Cleaned_dict[e]=Dict_[e]
    df = pd.DataFrame(data=Cleaned_dict)
    l.append(df)
    time.sleep(5)
df=pd.concat(l).reset_index(drop=True)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32


C:\Users\DingR\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [44]:
df=df.sort_values(by=['Timestamp'])
#df.to_csv('load_1902.csv',index=False)

In [45]:
df

,AZPS,BCHA,CA ISO-TAC,IPCO,MWD-TAC,NEVP,PACE,PACW,PGE,PGE-TAC,PSEI,SCE-TAC,SDGE-TAC,Timestamp,VEA-TAC
18,3182,NaN,25277,NaN,NaN,3398,5209,2563,NaN,10120,3556,11202,1881,2017-03-01T08:00:00-00:00,52
13,2502,NaN,20647,NaN,NaN,3968,5631,2284,NaN,9777,3475,9333,2741,2017-03-01T09:00:00-00:00,51
10,2516,NaN,24092,NaN,NaN,3499,4656,3073,NaN,9605,3734,10772,2332,2017-03-01T10:00:00-00:00,75
23,2771,NaN,20600,NaN,NaN,3632,5141,2439,NaN,9644,3715,10648,1750,2017-03-01T11:00:00-00:00,78
17,2605,NaN,22805,NaN,NaN,3568,5347,2790,NaN,9775,3580,11136,2002,2017-03-01T12:00:00-00:00,54
15,3074,NaN,23176,NaN,NaN,3358,5550,2658,NaN,10452,3708,12524,2140,2017-03-01T13:00:00-00:00,72
7,2343,NaN,23822,NaN,NaN,3463,4644,2366,NaN,11699,3103,10698,2637,2017-03-01T14:00:00-00:00,67
0,2888,NaN,20984,NaN,NaN,3305,4735,3034,NaN,12197,2683,11538,2717,2017-03-01T15:00:00-00:00,66
12,3162,NaN,24138,NaN,NaN,3398,5377,2632,NaN,11774,3038,11469,1759,2017-03-01T16:00:00-00:00,96
22,2979,NaN,25773,NaN,NaN,3723,5612,2922,NaN,11565,3323,10738,2465,2017-03-01T17:00:00-00:00,63


In [47]:
df.to_csv('17-19load.csv',index=False)